# The ExpressionExpert.ipynb

## Introduction

The ExpressionExpert.ipynb is a Jupyter notebook for the analysis of promoter libraries. The user uploads one or several sequence libraries with associated expression values. The notebook guides:
 1. the statistical analysis, 
 2. the training of a random forest regressor,
 3. the evaluation of the regressor performance,
 4. the set-up of a synthetic promoter library within the experimental exploration region, and 
 5. the selection of new promoters with defined expression.
 
--- 
## Workflow initialization
The workflow is distributed in different notebooks. However, many functions call the same file and will store results in a common directory. In the following a project directory is generated in combination with the file name of the promoter library. These informations are stored in the 'config.csv' file for all other notebooks. You need to run the initialization of the folder structure and the generation of the config-file once before starting with the subsequent notebooks. Once initialized, you can also rename plots or names by changing the names in the config-file and then re-running notebooks 1-5.

**User Input:** <br>
 * **Change 'Data_File' to the file name of your promoter library in the Notebook directory. Import external files if necessary.**
 * **Input the column name of the expression values in variable 'Sequence_column'.**

*Example: <br>
An example file for a promoter library in* Pseudomonas putida *KT2440 is provided.* <br>
Data_File = 'Example1_Pput.csv'

---
**Author: Ulf Liebal** <br>
**Institution: Institute of Applied Microbiology, RWTH Aachen** <br>
**Contact: ulf.liebal@rwth-aachen.de** <br>
**Date: 2019/12/19** <br>

---

In [ ]:
import os
import time
#####################
# start User Input
# File name of the library, in csv format
# example input: 'Example1-Pput.csv', 'PromLib_EcolPtai.csv'
Data_File = 'meng.csv'  
# Use of common promoter sequence library,
# True, if a common promoter sequence library has been measured in different organisms (e.g. E. coli - P. taiwanensis)
# False, if different promoter sequence libraries are measured
Library_Sequence_single = True
# column name of expression activity
# example input: ['Promoter Activity'], ['Ecol Promoter Activity', 'Ptai Promoter Activity'],
Y_Col_Name = ['Ecol-expression']
# Promoter ID column name
# 'Strain ID'
ID_Col_Name = 'Sequence-ID'
# expression unit for plots
# example: '$\mu$M(GFP)/(gCDW*h)' 'GFP intensity'
Add_Feat = ['GC-content']
# column for additional features, this particular feature is generated by the script
Expression_Unit = 'relative-strength'
# Reference Sequence for sequence diversity histogram
# for X-Host 'GCCCATTGACAAGGCTCTCGCGGCCAGGTATAATTGCACG'
RefSeq = ''
# Kernal and function based machine learning approaches like SVM and artificial neural networks, require data standardization. Correlation and regression trees, including random forest, do not benefit from data standardization.
# Set variable with Boolean for data standardization
Data_Standard = False
# if previously a synthetic library was generated it is stored under a generic name with only the date as variable
# give synthetic library date to load file with default naming
SynLib_Date = time.strftime('%Y%m%d')
# Date for the random forest regression. 
ML_Date = time.strftime('%Y%m%d')
# Test set cut-off, fraction of data removed from the original set to be used for quality assessment
TestRatio = .1
# Definition of the machine learning regression approach
# choose 'RFR': random forest regression, 'SVR': support vector regression, 'TPOT': TPOT based automated tree pipeline (installation required)
ML_Regressor = 'SVR'
# Parameters for the synthetic promoter library
# the 'Sequence_Distance_cutoff' determines how distant are sequences allowed to be different from the most common sequence reference
# The statistical analysis gives a histogram of sequence distances, take an appropriate value from there
# in Example1-Pput: 0.11
Sequence_Distance_cutoff = .11
# not all positions might be sampled with all nucleotides, the parameter 'Entropy_cutoff' determines the minimum position diversity
# The position diversity entropy-bargraph gives an indication on the right parameter choice.
# in Example1-Pput: 0.15
Entropy_cutoff = .2
# decide on how many synthetic promoters you want to simulate
Synth_Seq_MaxNumber = 10000

# end User Input
#####################


# extract the filename for naming of newly generated files
File_Base = Data_File.split('.')[0]
# the generated files will be stored in a subfolder with custom name
Data_Folder = 'data-{}'.format(File_Base)
try:
    os.mkdir(Data_Folder)
    print('Data directory ', Data_Folder, 'created.')
except FileExistsError:
    print('Already existent data directory ', Data_Folder, '.')

# Definition of names
# make specific changes prefereably in the user input space above.
Name_Dict = {
    'Data_File': Data_File,
    # use of single sequence library for different expression measurements, 
    # if 'True' then only one sequence column should be in the csv
    'Library_Sequence_single': Library_Sequence_single,
    # column name of sequence library
    # if distinct sequence libraries were used for different expression measurement, then put names in a vector ['sequence1', 'sequence2']
    'Sequence_column': 'Sequence',
    # column name for the expression strength
    # if more library expression measurements are conducted, then put Y_Col_Name in a vector ['name1', 'name2']
    'Y_Col_Name': Y_Col_Name, # ['Promoter Activity'], ['Ecol Promoter Activity', 'Ptai Promoter Activity'],
    # Promoter ID column name
    'ID_Col_Name': ID_Col_Name,
    # number of separate promoter libraries expression measurements
    'Library_Expression': len(Y_Col_Name),
    # column name for additional features input
    'Add_Feat': Add_Feat
    # set the desired figure file type here, e.g. svg, png, pdf
    'Figure_Type': 'png',
    # base file naming
    'File_Base': File_Base,
    'Data_Folder': Data_Folder,
    'HM_File': 'Plot_PositionNucleotideStats',
    'SampSeqDist_File': 'Plot_SampleSequenceDistance',
    'Entropy_File': 'Plot_PositionEntropy',
    'SamplingDiv_File': 'Plot_SamplingDiversity',
    'ExprHist_File': 'Plot_ExpressionHist',
    'LogoPlot_File': 'Plot_Logo-FeatureImportance',
    'CorrPlot_File': 'Plot_Corr-Ytrue-VS-Ypred',
    'Csv_ID': 'Synth-Library',
    'X_Expr': 'Plot_CrossExpr',
    'RefSeq': RefSeq,
    'ML_Regressor': ML_Regressor,
    'Data_Standard': Data_Standard,
    'SynLib_Date': SynLib_Date,
    'ML_Date': ML_Date,
    # Machine learning files
    'TestRatio': TestRatio,
    'Expression_Unit': Expression_Unit,
    # Parameters for the synthetic library
    'Sequence_Distance_cutoff': Sequence_Distance_cutoff,
    'Entropy_cutoff': Entropy_cutoff,
    'Synth_Seq_MaxNumber': Synth_Seq_MaxNumber
}

# constructing the config.txt file
with open('config.txt', 'w') as f:
    print('# This file contains the naming conventions for all output files. It is automatically generated when going through step "0-Workflow".', file=f)
    for key, value in Name_Dict.items():
        print('{}: {}'.format(key, value), file=f)


---
 
## Statistical analysis 

The statistical analysis provides an overview to important properties of the promoter library. It visualizes the distribution of measured expression strength with a histogram. This is particularly informative if several promoter libraries are analysed in order to compare the coverage of expression strengths. It is important to delineate the sequence exploration space that has been spanned with the promoter library. The sequence exploration space defines the allowed sequence input for prediction of expression strength and limits the sequences for the generation of the synthetic promoter library. The exploratory space is identified by two properties: 
 * the promoter library diversity, calculated by the percentage of nucleotide changes over the full sequence for all samples in the library, and 
 * the position specific diversity, i.e. how many different nucleotides are sampled on each position, determined by the entropy on each position.

The promoter diversity is visualized with a histogram of the frequency of sequences with a given number of nucleotide exchanges. The histogram shows how much nucleotides need to be changed to mutually convert sequences, and what is the maximum and average nucleotide differences. The entropy measurement of the position diversity informs about how many nucleotides have been sampled for each position.

The promoter library with associated expression activities contains information how each nucleotide-position contributes to expression. This information is extracted by calculating the average and the variance of expression on each nucleotide position. The output is a heat-map that shows which positions are on average associated with higher or lower expression.

[1-Statistical-Analysis.ipynb](./1-Statistical-Analysis.ipynb)

---

## Regressor training
For random-forest regression, the data is split into a training and a test set, by default with a ratio of 9:1. The expression values are scaled to zero mean and unit variance based on the training set. Positions that are non-informative because no alternative nucleotides have been tested are deleted. The performance evaluation is based on the R^2 score from sklearn. The correlation of measured and predicted expression values is plotted. The feature importance from the random forest regression represent the contributions of each nucleotide-position to the prediction. They are extracted and visualized with a Logo-plot.

[2-Regressor-Training.ipynb](./2-Random-Forest-Training.ipynb)

---

## Evaluation of regressor performance
The goodness of a regressor is evaluated using the coefficient of correlation, R^2, and root mean squared error. Moreover, the trained feature importances from the random forest is visualized as a sequence logo to allow comparison with knwon regulatory elements.

[3-Regressor-Performance.ipynb](./3-Regressor-Performance.ipynb)

---

## Generation of random sequences in the exploration space
The generation of random sequences covered by the experimentally tested sequences serves to delineate the whole expected range of expression strength and to select for novel promoters with defined activity. The sequence exploration space is defined by the distance of nucleotide exchanges to a reference sequence and the information content for each nucleotide on all positions. Only if a position along the sequence has a sufficient information content can the conclusion be drawn that enough input diversity was present to learn a correct output mapping. The output mapping is determined with the position specific entropy and the user decides about the threshold of the entropy such that a position is being used for random sequence construction. The reference sequence can be pre-determined by the user or automatically assembled from the most common nucleotides at each position. The user has to identify the appropriate distance that the random sequences are allowed to be different from the reference. If the available sequences within the allowed exploration space is smaller than a threshold, all sequences will be generated, otherwise random sequences will be generated that satisfy the exploration space conditions. 

[4-Exploration-Space.ipynb](./4-Exploration-Space.ipynb)


## Software dependencies

CPython 3.7.3<br>
IPython 7.6.1<br>
<br>
ipywidgets 7.5.0<br>
matplotlib 3.1.0<br>
numpy 1.16.4<br>
pandas 0.24.2<br>
sklearn 0.21.2<br>
scipy 1.2.1<br>
logomaker 0.8<br>
joblib 0.13.2<br>
<br>
compiler   : MSC v.1915 64 bit (AMD64)<br>
system     : Windows<br>
release    : 10<br>
machine    : AMD64<br>
processor  : AMD64 Family 23 Model 8 Stepping 2, AuthenticAMD<br>
CPU cores  : 16<br>
interpreter: 64bit<br>

%load_ext watermark
%watermark -v -m -p ipywidgets,matplotlib,numpy,pandas,sklearn,logomaker